In [1]:
# Install required packages
!pip install transformers==4.41.0
!pip install peft==0.11.0
!pip install datasets==2.19.0
!pip install trl==0.8.6
!pip install bitsandbytes==0.43.1
!pip install accelerate==0.30.1
!pip install torch==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 63.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 75.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# from google.colab import userdata
# hf_token = userdata.get('HF_TOKEN')

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf-token")


from huggingface_hub import login
login(token=secret_value_0)

In [3]:
import os
import torch
import gc
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from transformers.pipelines import pipeline
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer
import pandas as pd
import json

# Enhanced memory management
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch detected {torch.cuda.device_count()} GPU(s)")

2025-06-21 09:18:34.834644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750497515.054685      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750497515.118138      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PyTorch detected 2 GPU(s)


## Dataset Preparation

In [4]:
def load_financial_datasets():
    """Load and combine financial datasets - reduced size for faster training"""
    
    # Load Finance Alpaca dataset
    try:
        finance_alpaca = load_dataset("gbharti/finance-alpaca", split="train")
        print(f"Loaded Finance Alpaca: {len(finance_alpaca)} samples")
        
        # Limit dataset size for faster training (5000 samples max)
        if len(finance_alpaca) > 10000:
            finance_alpaca = finance_alpaca.select(range(10000))
            print(f"Limited dataset to 5000 samples for faster training")
            
    except:
        print("Finance Alpaca not available, creating sample dataset")
        # Create sample financial data if dataset not accessible
        sample_data = [
            {
                "instruction": "What is the P/E ratio and how is it calculated?",
                "input": "",
                "output": "The P/E ratio (Price-to-Earnings ratio) is calculated by dividing the market price per share by the earnings per share (EPS). It indicates how much investors are willing to pay for each dollar of earnings."
            },
            {
                "instruction": "Explain the importance of SEC 10-K filings for investors.",
                "input": "",
                "output": "SEC 10-K filings are annual reports that provide a comprehensive overview of a company's business, financial condition, and results of operations. They are crucial for investors as they contain audited financial statements, risk factors, and management discussion."
            },
            {
                "instruction": "What are the key financial ratios for stock analysis?",
                "input": "",
                "output": "Key financial ratios include P/E ratio, P/B ratio, ROE, ROA, debt-to-equity ratio, current ratio, and profit margins. These help evaluate profitability, liquidity, and financial health."
            },
            {
                "instruction": "How do interest rates affect stock valuations?",
                "input": "",
                "output": "Higher interest rates increase the discount rate for future cash flows, reducing present value of stocks. They also increase borrowing costs and may slow economic growth, negatively impacting stock prices."
            }
        ] * 1000  # Replicate for training
        finance_alpaca = Dataset.from_list(sample_data)

    return finance_alpaca

def format_financial_dataset(dataset):
    """Format dataset for instruction tuning with TinyLlama format"""

    def format_prompt(example):
        # TinyLlama chat format[1]
        if example.get("input", "").strip():
            prompt = f"<|system|>\nYou are a helpful financial analyst assistant.\n<|user|>\n{example['instruction']}\n{example['input']}\n<|assistant|>\n{example['output']}"
        else:
            prompt = f"<|system|>\nYou are a helpful financial analyst assistant.\n<|user|>\n{example['instruction']}\n<|assistant|>\n{example['output']}"
        return {"text": prompt}

    formatted_dataset = dataset.map(format_prompt)
    return formatted_dataset

# Load and format dataset
financial_dataset = load_financial_datasets()
formatted_dataset = format_financial_dataset(financial_dataset)
print(f"Formatted dataset size: {len(formatted_dataset)}")

Generating train split:   0%|          | 0/68912 [00:00<?, ? examples/s]

Loaded Finance Alpaca: 68912 samples
Limited dataset to 5000 samples for faster training


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Formatted dataset size: 10000


## Model and Tokenizer Configuration

In [5]:
# Model configuration for TinyLlama - optimized for speed
BASE_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
NEW_MODEL_NAME = "TinyLlama-1.1B-Financial-LoRA"

def setup_model_and_tokenizer():
    """Setup quantized TinyLlama model and tokenizer with optimized configuration"""

    # Optimized quantization configuration for faster training
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,  # Use FP16 for T4 GPUs
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_storage=torch.uint8,
    )

    # Load TinyLlama tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL_NAME,
        trust_remote_code=True,
        use_fast=True
    )
    
    # TinyLlama tokenizer configuration
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Load TinyLlama model with optimized quantization
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,  # Use FP16 for T4 GPUs
        low_cpu_mem_usage=True,
    )

    # Disable gradient checkpointing for speed (we have enough memory with TinyLlama)
    # model.gradient_checkpointing_enable()

    # Prepare model for k-bit training - essential for QLoRA
    model = prepare_model_for_kbit_training(model)

    # Disable caching for training
    model.config.use_cache = False

    return model, tokenizer

model, tokenizer = setup_model_and_tokenizer()
print("TinyLlama model and tokenizer loaded successfully")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

TinyLlama model and tokenizer loaded successfully


## LoRA Configuration

In [6]:
def create_tinyllama_lora_config():
    """Create LoRA configuration optimized for TinyLlama architecture and speed"""

    # TinyLlama-specific LoRA configuration optimized for speed[2]
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,  # Reduced rank for faster training
        lora_alpha=16,  # Proportional to rank
        target_modules=[
            "q_proj", "v_proj",  # Target only essential attention layers for speed
        ],
        lora_dropout=0.05,  # Reduced dropout
        bias="none",
        use_rslora=True,  # Enable RSLoRA for better efficiency
        inference_mode=False,
    )

    return lora_config

# Apply LoRA to TinyLlama model
lora_config = create_tinyllama_lora_config()
model = get_peft_model(model, lora_config)

# Print trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"Trainable params: {trainable_params:,}")
    print(f"All params: {all_param:,}")
    print(f"Trainable%: {100 * trainable_params / all_param:.2f}%")

print_trainable_parameters(model)

Trainable params: 1,126,400
All params: 616,732,672
Trainable%: 0.18%


## Training Configuration

In [7]:
def create_fast_training_arguments():
    """Create training arguments optimized for speed"""

    training_args = TrainingArguments(
        output_dir=f"./results/{NEW_MODEL_NAME}",
        per_device_train_batch_size=8,  # Increased batch size for TinyLlama
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,  # Reduced for faster training
        num_train_epochs=2,  # Single epoch often sufficient for financial tasks
        max_steps=500,  # Limit total steps for speed
        learning_rate=2e-4,  # Higher learning rate for faster convergence
        fp16=True,  # Use FP16 for T4 GPUs
        bf16=False,
        logging_steps=25,
        save_steps=250,
        evaluation_strategy="steps",
        eval_steps=250,
        warmup_steps=25,  # Reduced warmup
        lr_scheduler_type="linear",
        optim="adamw_torch",  # Faster than paged_adamw_8bit
        weight_decay=0.01,
        max_grad_norm=0.3,
        group_by_length=True,
        dataloader_pin_memory=True,
        dataloader_num_workers=2,
        remove_unused_columns=False,
        gradient_checkpointing=False,  # Disabled for speed
        report_to=None,
        save_total_limit=2,  # Limit saved checkpoints
        load_best_model_at_end=False,  # Skip for speed
    )

    return training_args

training_args = create_fast_training_arguments()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
import wandb
wandb.login(key='dceb696a48075fb2e7a8039ee03e6064acdfa5b4')


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thetilakgupta (thetilakgupta-Indian Institute of Technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Training Setup and Execution

In [9]:
def create_tinyllama_trainer(model, tokenizer, dataset, training_args, lora_config):
    """Create SFT trainer optimized for TinyLlama and speed"""

    # Use smaller eval dataset for speed
    eval_size = min(100, len(dataset) // 10)
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        eval_dataset=dataset.select(range(eval_size)),
        peft_config=lora_config,
        dataset_text_field="text",
        tokenizer=tokenizer,
        args=training_args,
        max_seq_length=1024,  # Reduced from 2048 for speed[3]
        packing=True,  # Enable packing for efficiency
        dataset_kwargs={
            "add_special_tokens": False,
            "append_concat_token": False,
        }
    )

    return trainer

# Create trainer
trainer = create_tinyllama_trainer(model, tokenizer, formatted_dataset, training_args, lora_config)

# Memory cleanup before training
torch.cuda.empty_cache()
gc.collect()

# Start training
print("Starting TinyLlama financial fine-tuning...")
print(f"Memory allocated before training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
trainer.train()

# Save the fine-tuned model
trainer.save_model()
print(f"Model saved to ./results/{NEW_MODEL_NAME}")

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2310 > 2048). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


Starting TinyLlama financial fine-tuning...
Memory allocated before training: 0.37 GB


...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:464

Step,Training Loss,Validation Loss
250,2.237600,2.279014
500,2.238300,2.269315


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Model saved to ./results/TinyLlama-1.1B-Financial-LoRA


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Model Testing and Inference

In [10]:
def test_tinyllama_financial_model(model, tokenizer):
    """Test the fine-tuned TinyLlama model on financial queries"""

    # Test prompts in TinyLlama format[1]
    test_prompts = [
        "<|system|>\nYou are a helpful financial analyst assistant.\n<|user|>\nExplain what investors should look for in a company's 10-K filing.\n<|assistant|>\n",
        "<|system|>\nYou are a helpful financial analyst assistant.\n<|user|>\nWhat are the key financial ratios for evaluating a stock?\n<|assistant|>\n",
        "<|system|>\nYou are a helpful financial analyst assistant.\n<|user|>\nHow do interest rate changes affect stock market valuations?\n<|assistant|>\n"
    ]

    # Create pipeline for inference with TinyLlama
    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=150,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        torch_dtype=torch.float16
    )

    print("Testing fine-tuned TinyLlama financial model:")
    print("=" * 50)

    for i, prompt in enumerate(test_prompts, 1):
        print(f"\nTest {i}:")
        question = prompt.split("<|user|>\n")[1].split("\n<|assistant|>")[0]
        print("Input:", question)

        result = pipe(prompt)
        full_response = result[0]['generated_text']
        
        # Extract model response
        if "<|assistant|>\n" in full_response:
            response = full_response.split("<|assistant|>\n")[-1]
            if "<|" in response:
                response = response.split("<|")[0]
        else:
            response = full_response

        print("Output:", response.strip()[:200] + "..." if len(response.strip()) > 200 else response.strip())
        print("-" * 30)

# Test the model
test_tinyllama_financial_model(model, tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

Testing fine-tuned TinyLlama financial model:

Test 1:
Input: Explain what investors should look for in a company's 10-K filing.
Output: The 10-K is a document that is filed with the SEC.  It is a legal document that is required by law to be filed with the SEC.  It is a legal document that is required by law to be filed with the SEC.  ...
------------------------------

Test 2:
Input: What are the key financial ratios for evaluating a stock?
Output: I'm not sure if this is the right answer, but I'm going to give it a shot.  I'm not sure if this is the right answer, but I'm going to give it a shot.  I'm not sure if this is the right answer, but I'...
------------------------------

Test 3:
Input: How do interest rate changes affect stock market valuations?
Output: The short answer is that it doesn't.  The long answer is that it does.  The short answer is that it doesn't.  The long answer is that it does.  The short answer is that it doesn't.  The long answer is...
----------------------

## Model Saving and Loading

In [11]:
def save_and_load_tinyllama_model():
    """Save complete model for Ollama and demonstrate loading"""

    # STEP 1: Save LoRA adapter (your current approach)
    model.save_pretrained(f"./saved_models/{NEW_MODEL_NAME}")
    tokenizer.save_pretrained(f"./saved_models/{NEW_MODEL_NAME}")
    print(f"LoRA adapter saved to ./saved_models/{NEW_MODEL_NAME}")

    # STEP 2: Create complete merged model for Ollama
    print("Creating complete model for Ollama...")
    
    # Merge LoRA weights with base model
    merged_model = model.merge_and_unload()
    
    # Save the complete merged model (includes config.json and all files)
    merged_model.save_pretrained(f"./ollama_models/{NEW_MODEL_NAME}")
    tokenizer.save_pretrained(f"./ollama_models/{NEW_MODEL_NAME}")
    
    print(f"✅ Complete model saved to ./ollama_models/{NEW_MODEL_NAME}")
    print("This model contains all files needed for Ollama (including config.json)")

    # STEP 3: Demonstrate loading for Mac compatibility test
    print("Loading model for Mac compatibility test...")
    
    # Load base TinyLlama model
    base_model_reload = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        ),
        device_map="auto",
        torch_dtype=torch.float16,
    )

    # Load LoRA weights
    financial_tinyllama_model = PeftModel.from_pretrained(
        base_model_reload,
        f"./saved_models/{NEW_MODEL_NAME}"
    )

    print("TinyLlama model successfully reloaded with LoRA weights")
    
    # STEP 4: Create Ollama Modelfile
    create_ollama_modelfile()
    
    return financial_tinyllama_model, merged_model

def create_ollama_modelfile():
    """Create Modelfile for Ollama deployment"""
    
    modelfile_content = f"""FROM ./ollama_models/{NEW_MODEL_NAME}

TEMPLATE \"\"\"<|system|>
You are a helpful financial analyst assistant.
<|user|>
{{{{ .Prompt }}}}
<|assistant|>
\"\"\"

PARAMETER stop "<|user|>"
PARAMETER stop "<|system|>"
PARAMETER temperature 0.1
PARAMETER top_p 0.9
PARAMETER num_ctx 2048
"""
    
    with open("Modelfile", "w") as f:
        f.write(modelfile_content)
    
    print("✅ Modelfile created for Ollama")
    print("\n" + "="*50)
    print("OLLAMA DEPLOYMENT INSTRUCTIONS:")
    print("="*50)
    print("1. Download the './ollama_models/' folder to your local Mac")
    print("2. Download the 'Modelfile' to the same directory")
    print("3. Open terminal in that directory")
    print("4. Run: ollama create financial-analyst -f Modelfile")
    print("5. Run: ollama run financial-analyst")
    print("="*50)

# Save the model (updated call)
final_model, complete_model = save_and_load_tinyllama_model()


LoRA adapter saved to ./saved_models/TinyLlama-1.1B-Financial-LoRA
Creating complete model for Ollama...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


✅ Complete model saved to ./ollama_models/TinyLlama-1.1B-Financial-LoRA
This model contains all files needed for Ollama (including config.json)
Loading model for Mac compatibility test...
TinyLlama model successfully reloaded with LoRA weights
✅ Modelfile created for Ollama

OLLAMA DEPLOYMENT INSTRUCTIONS:
1. Download the './ollama_models/' folder to your local Mac
2. Download the 'Modelfile' to the same directory
3. Open terminal in that directory
4. Run: ollama create financial-analyst -f Modelfile
5. Run: ollama run financial-analyst


In [12]:
# Final memory cleanup
torch.cuda.empty_cache()
gc.collect()
print("Training completed successfully!")
print(f"Expected training time: 4-6 hours on Kaggle T4 GPU")
print(f"Model size: ~1.5GB (suitable for Mac deployment)")

Training completed successfully!
Expected training time: 4-6 hours on Kaggle T4 GPU
Model size: ~1.5GB (suitable for Mac deployment)
